In [ ]:
import json
from bert_score import BERTScorer
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import *
import spacy
import seaborn as sns
import random
import re

from nltk.tokenize import treebank #ptb tokenizer to convert for DAE stuff
ptb_tok = treebank.TreebankWordTokenizer()
detok = treebank.TreebankWordDetokenizer()


In [ ]:

stemmer = PorterStemmer()
nlp = spacy.load("en_core_web_sm")
scorer = BERTScorer(lang="en", rescale_with_baseline=False, model_type="microsoft/deberta-xlarge-mnli")
rng = np.random.default_rng(19680801)

df = pd.read_csv('./maynez_docset.csv')

# opening the json file
with open('subgraph_meta/data.json') as json_file:
    data = json.load(json_file)

In [ ]:
# only pair, with context, with after, all
OPTIONS  = ['OP', 'C', 'AF', 'ALL']

# method that gets a indexed lists for the pairs, ready for scoring
def get_graph_pairs(pairs, option):
    l1 = []
    l2 = []
    for p in pairs:
        s1 = p['s1']
        s2 = p['s2']
        if (option == 'C') or (option == 'ALL'):
            s1 = p['context']+s1
            s2 = p['context']+s2
        if (option == 'AF') or (option == 'ALL'):
            s1 = s1+p['afterstr']
            s2 = s2+p['afterstr']
        if (s1.strip() == "") or (s2.strip() == ""):
            continue
        # this is a reverse of another example
        if s1 in l2:
            if l1[l2.index(s1)] == s2:
                continue
        l1.append(s1)
        l2.append(s2)
    return l1, l2


In [ ]:
keylist = list(data.keys())

def score_graph_ind(ind, tech):
    strings1, strings2 = get_graph_pairs(data[keylist[ind]], tech)
    try:
        P, R, F1 = scorer.score(strings1, strings2)
        #print("ID: "+keylist[ind])
        #print(F1.mean())
        return F1
    except:
        return 

def get_all_graphdata(option):
    f1s = []
    for i in range(0, len(data.keys())):
        g = score_graph_ind(i, option)
        tmp = {}
        tmp['id'] = i
        tmp['f1mat'] = g
        if g != None:
            f1s.append(tmp)
        else:
            tmp['f1mat'] = [-1];
            f1s.append(tmp)
    return f1s

In [ ]:
keylist = list(data.keys())

def vis_score_graph(ind, option):
    
    return vis_score_id(keylist[ind], option)

def get_context(idval, dictkey):
    newid = int(idval)
    tmp = df.loc[df['bbcid']==newid]
    return tmp[dictkey][tmp.first_valid_index()]

def simcount(scores):
    simcount = 0
    THRESH = .7
    for s in scores:
        if s>THRESH:
            simcount+=1
    return simcount, len(scores)

stop_words = set(stopwords.words('english'))

def filter_stopwords(sraw):
    word_tokens = word_tokenize(sraw)
    
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def check_content (filt1, filt2):
    stem1 = [stemmer.stem(f) for f in filt1]
    stem2 = [stemmer.stem(f) for f in filt2]
    if stem1==stem2:
        return True

def loose_stem(filt1, filt2):
    stem1 = [stemmer.stem(f) for f in filt1]
    stem2 = [stemmer.stem(f) for f in filt2]
    for s in stem1:
        if s in stem2:
            return True

def count_stop_stem(list1, list2):
    stopmatches = []
    stemmatches = []
    loosematches = []
    for i in range (0, len(list1)):
        t1 = filter_stopwords(list1[i].replace("\"", ""))
        t2 = filter_stopwords(list2[i].replace("\"", ""))
        if t1==t2:
            stopmatches.append(i)
        elif check_content(t1, t2):
            stemmatches.append(i)
        elif loose_stem(t1, t2):
            loosematches.append(i)
    return stopmatches, stemmatches, loosematches

def get_len_metrics(list1, list2):
    lens1 = []
    lens2 = []
    for i in range(0, len(list1)):
        lens1.append(len(list1[i].split()))
        lens2.append(len(list2[i].split()))
    lens1 = np.array(lens1)
    lens2 = np.array(lens2)
    avglen = (np.mean(lens1)+np.mean(lens2))/2
    avgdiff = np.mean(np.absolute(lens2-lens1))
    numzeros = len(lens1)+len(lens2) - (np.count_nonzero(lens1)+np.count_nonzero(lens2))
    return avglen, avgdiff, numzeros

def filter_stringlists (l1, l2):
    count = 0
    for i in range(0, len(l1)):
        if l1[i].split==l2[i].split():
            count+=1
            del l1[i]
            del l2[i]
    #print(str(count)+" WEIRD REPEATS REMOVED")
    return l1, l2

keylist = list(data.keys()) 


def vis_score_id(idv, option):
    ind = keylist.index(idv)
    exampleid = keylist[ind]
    # store metrics in dictionary
    res = {}
    res['id'] = exampleid
    res['context'] = get_context(idv, 'summary')

    #print("Context: \n"+get_context(idv, 'doc'))
    strings1, strings2 = get_graph_pairs(data[keylist[ind]], option)
    strings1, strings2 = filter_stringlists(strings1, strings2)
    try: 
        scores = score_graph_ind(ind, option)
        scount, tot = simcount(scores)
    except:
        scores = []
        scount, tot = -1, len(strings1)
    content, stem, loose_stem = count_stop_stem(strings1, strings2)
    #identical except for stopwords or quotations
    res['content_same'] = len(content)
    #identical content words after stemming (that doesn't overlap w/ content_same)
    res['stem_same'] = len(stem)
    res['loose_stem'] = len(loose_stem)
    #based on BERT-SCORE similarity
    res['similar'] = scount
    #metrics based on length
    res['avglen'], res['avgdiff'], res['numzeros'] = get_len_metrics(strings1, strings2)
    #total number of pairs
    res['total'] = tot
    res['scores'] = scores
    res['strings1'] = strings1
    res['strings2'] = strings2

    return res

def print_results(res):
    for i in range(0, len(res['strings1'])):
        print(str(res['scores'][i]))
        print(res['strings1'][i])
        print(res['strings2'][i])
        print("----------")

def get_all_results(option):
    reslist = []
    for i in range(0, len(keylist)):
        reslist.append(vis_score_graph(i, option))
    return reslist

#all_results = get_all_results('OP')
#all_res_df = pd.DataFrame(all_results)

In [ ]:
print_results(vis_score_graph(1, "AF"))

In [ ]:
def check_lemma(tok, d2):
    for d in d2:
        if tok.lemma_==d.lemma_:
            return True
    return False

def analyze_pos(ind, item):
    s1 = all_res_df.loc[ind]['strings1']
    s2 = all_res_df.loc[ind]['strings2']
    #print(s1[item])
    # print(s2[item])
    d1 = nlp(s1[item])
    d2 = nlp(s2[item])
    ncount1, vcount1 = 0, 0
    sharedn, sharedv = 0, 0
    for i in range(0, len(d1)):
        if d1[i].is_stop:
            continue
        if "VB" in d1[i].tag_:
            vcount1+=1
            if check_lemma(d1[i], d2):
                sharedv+=1
        elif "NN" in d1[i].tag_:
            ncount1+=1
            if check_lemma(d1[i], d2):
                sharedn+=1
    ncount2, vcount2 = 0, 0
    for i in range(0, len(d2)):
        if d2[i].is_stop:
            continue
        if "VB" in d2[i].tag_:
            vcount2+=1
        elif "NN" in d2[i].tag_:
            ncount2+=1
    vals = (sharedn, sharedv, vcount1, ncount1, vcount2, ncount2)
    res = []
    for v in vals:
        res.append(v)
    return res

def get_pos_info(ind):
    length =  len(all_res_df.loc[ind]['strings1'])
    if length==0:
        return None
    tot = analyze_pos(ind, 0)
    if length==1:
        return tot
    for i in range(1,length):
        tmp = analyze_pos(ind, i)
        for j in range(0, len(tot)):
            tot[j]+=tmp[j]
    for j in range(0, len(tot)):
            tot[j]=tot[j]/length
    return tot

pos_labels = ['sharedn', 'sharedv', 'vcount1', 'ncount1', 'vcount2', 'ncount2']
#takes long to run
def get_all_posinfo():
    res = []
    for i in range(0, all_res_df.shape[0]):
        res.append (get_pos_info(i))
    res = np.array(res)
    for i in range(0, len(res)):
        if res[i]==None:
            res[i]=[-1,-1,-1,-1,-1,-1]
    length = max(map(len, res))
    y=np.array([xi+[None]*(length-len(xi)) for xi in res])
    for i in range(0, len(pos_labels)):
        all_res_df[pos_labels[i]] = y[:, i]
        

In [ ]:
all_res_df = pd.read_json('subgraph_meta/processed.json')

In [ ]:
all_res_df

In [ ]:
#all_res_df['cont_prop'] = all_res_df['content_same'] / all_res_df['total']
#all_res_df['stemp_prop'] = all_res_df['stem_same'] / all_res_df['total']
#get_all_posinfo()
#all_res_df = all_res_df.fillna(-1)
#all_res_df.loc[all_res_df['avgdiff']==-1]

In [ ]:
#diff = all_res_df['ncount1']+all_res_df['ncount2']-all_res_df['sharedn']
small = all_res_df.loc[all_res_df['total']<70]
large = all_res_df.loc[all_res_df['total']>70]
sns.histplot(data=small['sharedn'])


In [ ]:

sns.histplot(data=large['sharedn'])

In [ ]:
#TODO come back to this later
def convert_alldata_df():
    res = []
    for a in alldata:
         res.append(np.mean(np.array(a['f1mat'])))
    return res
#all_res_df['sim_avgs'] = convert_alldata_df()

In [ ]:
#all_res_df.to_json('subgraph_meta/processed.json')

In [ ]:
def get_context_from_id(contextid):
    return df.loc[df['bbcid']==int(contextid)].iloc[0]['doc']

def tok_store_doc(doc, summ):
    tmp = {}
    tmp['doc'] = ptb_tok.tokenize(doc)
    tmp['doc'] = detok.detokenize(tmp['doc'])
    tmp['summ'] = ptb_tok.tokenize(summ)
    tmp['summ'] = detok.detokenize(tmp['summ'])
    return tmp

def get_contexts_fullsums():
    
    res1 = []
    res2 = []
    for index, row in all_res_df.iterrows():
        docstr = get_context_from_id(row['id'])
        for i in range(0, len(row['contexts'])):
            #temporarily removing contexts, after
            res1.append(tok_store_doc(docstr, row['s1s'][i]))#+row['afters'][i]))
            res2.append(tok_store_doc(docstr, row['s2s'][i]))#+row['afters'][i]))
        print(index)
    return res1, res2

#tok_tmp = ptb_tok.tokenize(text=all_res_df.loc[450]['afters'])
#newstr = detok.detokenize(tok_tmp)
#cont_fullsums = get_contexts_fullsums()
#write_dae_input(cont_fullsums[1], 'onlypath2')
#cont_fullsums[1]

In [ ]:
def write_dae_input(fullsum, name):
    with open(name+'.txt', 'w') as f:
        for full in fullsum:
            f.write(full['doc']+'\n')
            f.write(full['summ']+'\n\n')


In [ ]:
def clean_context(cont):
    cont = cont.replace("</s>", "")
    cont = cont.replace("-", "")
    return cont

def get_conts_afts():
    afters = []
    contexts = []
    s1s = []
    s2s = []
    for latid in all_res_df['id']:
        alist = []
        clist = []
        s1l = []
        s2l = []
        pathlist = data[str(latid)]
        for p in pathlist:
            alist.append(p['afterstr'])
            clist.append(clean_context(p['context']))
            s1l.append(clean_context(p['s1']))
            s2l.append(clean_context(p['s2']))
        afters.append(alist)
        contexts.append(clist)
        s1s.append(s1l)
        s2s.append(s2l)
    return afters, contexts, s1s, s2s

all_res_df['afters'], all_res_df['contexts'],all_res_df['s1s'], all_res_df['s2s']  = get_conts_afts()


In [ ]:
def read_dae_input(fn):
    linedata = []
    docdata = []
    i = 0
    with open(fn) as f:
        for line in f:
            if (i%3)==0:
                docdata.append(line)
            if (i-1)%3==0:
                linedata.append(line)
            i = i+1
    return linedata, docdata

fp1, docs1 = read_dae_input('fullpath1.txt')
fp2, docs2 = read_dae_input('fullpath2-shortened.txt')

In [ ]:
p1data = []
p2data = []
with open('path1-output.txt') as f:
    p1data = [json.loads(line) for line in f]

with open('path2-output.txt') as f:
    p2data = [json.loads(line) for line in f]

dae_df_1 = pd.DataFrame(p1data)
dae_df_2 = pd.DataFrame(p2data)

In [ ]:

def get_pred_avgs(predlist):
    avgs = []
    for plist in predlist:
        avgs.append(sum(plist)/len(plist))
    return avgs

# indicates roughly how different they are as a measure of differences of factual arcs 
# rationed per l1 
def get_fact_diffs(p1list, p2list):
    listlen = min(len(p1list), len(p2list))
    diffs = []
    for i in range(0, listlen):
        diffs.append(abs(sum(p2list[i])-sum(p1list[i]))/len(p1list[i]))
    return diffs

def get_conf_sum(clist):
    tot = 0
    for conf in clist:
        tot+=conf[1]
    return tot

def get_conf_diffs(c1list, c2list):
    listlen = min(len(c1list), len(c2list))
    diffs = []
    for i in range(0, listlen):
        sum1 = get_conf_sum(c1list[i])
        sum2 = get_conf_sum(c2list[i])
        diffs.append(abs(sum1-sum2)/((sum1+sum2)*0.5))
    return diffs

sns.histplot(data=get_conf_diffs(dae_df_1['probs'], dae_df_2['probs']))



In [ ]:
# random baseline sanity check for the bertscorer
def get_rand_stringlists(n):
    slist = []

    for i in range(0, n):
        try:
            docind = random.randint(0, len(all_res_df))
            tl = all_res_df.loc[docind]
            line = random.randint(0, len(tl['s1s']))
            slist.append(tl['contexts'][line]+tl['s1s'][line]+tl['afters'][line])
            #slist.append(tl['s1s'][line])
        except:
            i = i-1
    return slist

def do_random_baseline():
    l1 = get_rand_stringlists(1000)
    l2 = get_rand_stringlists(1000)
    l1 = l1[:len(l2)]
    l2 = l2[:len(l1)]
    print(len(l1))
    print(len(l2))
    P, R, F1 = scorer.score(l1, l2)
    return P, R, F1 
        

def update_df():
    global dae_df_2
    plist = []
    rlist = []
    f1list = []
    s1list = []
    s2list = []
    P, R, F1 = scorer.score(fp1[:len(fp2)], fp2[:len(fp2)])
    for index, row in dae_df_2.iterrows():
        plist.append(float(P[index]))
        rlist.append(float(R[index]))
        f1list.append(float(F1[index]))
        s1list.append(fp1[index])
        s2list.append(fp2[index])
    dae_df_2['P'], dae_df_2['R'], dae_df_2['F1'] = plist, rlist, f1list
    dae_df_2['s1'], dae_df_2['s2'] = s1list, s2list
#need to uncomment every time
update_df()

In [ ]:
dae_df_2

In [ ]:
s = "abcdef"
s[-1:]


In [ ]:
def get_str(ind, lis):
    res = lis[ind]
    for i in range(ind+1, len(lis)):
        res = res+" "+lis[i]
    return res

# physically get different path strings from full path
def find_indpaths(s1, s2):
    l1 = s1.split()
    l2 = s2.split()
    start = 0
    for i in range(0, min(len(l1), len(l2))):
        if (l1[i]==l2[i]):
            continue
        start = i
        break
    l1 = l1[start:]
    l2 = l2[start:]
    for i in range(1, min(len(l1), len(l2))+1):
        if l1[-i]==l2[-i]:
            ""
        else:
            l1 = l1[:(-i+1)]
            l2 = l2[:(-i+1)]
            break
    
    #if len(l1)>1 or len(l2)>1:
        #print(l1)    
    return get_str(0, l1), get_str(0, l2)

        
def get_clean_table():
    alldata = []
    dae_a_list = dae_df_1['preds']
    dae_b_list = dae_df_2['preds']
    p = dae_df_2['P']
    r = dae_df_2['R']
    f1 = dae_df_2['F1']
    for i in range(0, len(fp2)):
        tmp = {}
        tmp['doc'] = docs1[i]
        assert docs1[i]==docs2[i]
        tmp['full_a'] = fp1[i]
        tmp['full_b'] = fp2[i]
        tmp['invalid'] = False
        if tmp['full_a']==tmp['full_b']:
            print("BAD")
            tmp['invalid'] = True
        tmp['short_a'], tmp['short_b'] = find_indpaths(tmp['full_a'], tmp['full_b']) 
        tmp['dae_a'] = dae_a_list[i]
        tmp['dae_b'] = dae_b_list[i]
        tmp['precision'] =  p[i]
        tmp['recall'] = r[i]
        tmp['f1']=  f1[i]
        alldata.append(tmp)
    return pd.DataFrame(alldata)
        



table = get_clean_table()

In [ ]:
(table.loc[table['invalid']==False]).to_csv("pathdata_table.csv", )

In [ ]:
table = pd.read_csv('pathdata_table.csv')

In [ ]:
def clean_str(slist):
    for i in range(0, len(slist)):
        slist[i] = slist[i].replace("\n","" )
    slist = [line.strip() for line in slist]
    return slist

def clean_bertscore_out(table):
    d = clean_str(table['doc'][:1])
    fa = [clean_str(table['full_a'][:10])]
    fb = [clean_str(table['full_b'][:10])]
    return scorer.score(d, fa)

clean_bertscore_out(table)

In [ ]:
#print(sum(P)/len(P))
#print(sum(R)/len(R))
#print(sum(F1)/len(F1))
sentsum = dae_df_2['sent']+dae_df_1['sent'][:len(dae_df_2)]
sns.scatterplot(x=dae_df_2['F1'], y=sentsum)

In [ ]:

from scipy.stats import spearmanr
from scipy.stats import pearsonr


spearmanr(np.array(dae_df_2['F1']), np.array(get_fact_diffs(dae_df_1['preds'], dae_df_2['preds'])))

In [ ]:
dae_df_2

In [ ]:
rand_base = do_random_baseline()

In [ ]:
dae_df_2

In [ ]:
print(sum(rand_base[0])/len(rand_base[0]))
print(sum(rand_base[1])/len(rand_base[1]))
print(sum(rand_base[2])/len(rand_base[2]))


In [ ]:

s = "fat boy."
re.sub(r"[^a-zA-Z0-9]+", ' ', s).split()

In [ ]:
dae_df_1.loc[0]['arcs'][1]

In [126]:
def affected_arcs(row, prunedletter):
    refdf = dae_df_1
    if prunedletter =='b':
        refdf = dae_df_2
    allarcs = refdf.loc[row['oldid']]['arcs']
    pathwords = []
    pathwords = re.sub(r"[^a-zA-Z0-9]+", ' ',row['short_'+prunedletter] ).split()
    affected = {}
    ind = 0
    #print(allarcs)
    for arc in allarcs:
        for p in pathwords:
            if p in arc:
                affected[arc] = int(row['dae_'+prunedletter][1+ind*3])
                break
        ind+=1
    return affected
    
def check_hallucination(row):
    pwordsa = re.sub(r"[^a-zA-Z0-9]+", ' ',row['short_a'] ).split()
    pwordsb = re.sub(r"[^a-zA-Z0-9]+", ' ',row['short_b'] ).split()
    halluc = ''
    #print(pwordsb)
    #print(row['doc'])
    for a in pwordsa:
        if a not in row['doc']:
            halluc = halluc+'a'
            break
            
    for b in pwordsb:
        if b not in row['doc']:
            halluc = halluc+'b'
            break
    return halluc
    

# go through table, see which paths would get pruned / left if 
def see_dae_pruned():
    pruned = []
    table['oldid'] = table['Unnamed: 0']
    table['graphind'] = table.groupby('doc', sort=False).ngroup()
    for index, row in table.iterrows():
        acnt = (row['dae_a']).count('0')
        bcnt = (row['dae_b']).count('0')
        tmp = {}
        tmp = row
        if acnt < bcnt:
            tmp['dae_pruned'] = 'b'
        elif bcnt < acnt:
            tmp['dae_pruned'] = 'a'
        else:
            tmp['dae_pruned'] = '0'
            
        tmp['halluc'] = check_hallucination(row)
        #if tmp['dae_pruned'] == '0' and tmp['halluc'] == '':
        #    continue
        #if tmp['dae_pruned']=='a' or 'a' in tmp['halluc']:
        tmp['arcs_a'] = affected_arcs(row, 'a')
        #if tmp['dae_pruned']=='a' or 'a' in tmp['halluc']:
        tmp['arcs_b'] = affected_arcs(row, 'b')
        pruned.append(tmp)
        
    ret = pd.DataFrame(pruned)
    ret.drop(columns=['Unnamed: 0'])
    return ret
pruned = see_dae_pruned()
pruned = pruned.drop(columns=['Unnamed: 0'])

In [127]:
dae_prune = pruned.loc[pruned['dae_pruned']!='0']

In [123]:
def dict_avg (arcs):
    total = 0
    for a in arcs.keys():
        total+=arcs[a]
    length = len(list(arcs.keys()))
    if length==0:
        return -1
    return float(total)/length

def zero_cnt (arcs):
    total = 0
    for a in arcs.keys():
        if arcs[a]==0:
            total+=1
    length = len(list(arcs.keys()))
    if length==0:
        return -1
    return float(total)

def get_dae_arc_average(row):
    cur = row['dae_pruned']
    other = 'a'
    if cur =='a':
        other = 'b'
    curarcs = row['arcs_'+cur]
    otherarcs = row['arcs_'+other]
    return dict_avg(curarcs), dict_avg(otherarcs)

def get_dae_arc_coverage(row):
    cur = row['dae_pruned']
    other = 'a'
    if cur =='a':
        other = 'b'
    curarcs = row['arcs_'+cur]
    otherarcs = row['arcs_'+other]
    curcnt = row['dae_'+cur].count('0')
    othcnt = row['dae_'+other].count('0')
    if othcnt ==0:
        return zero_cnt(curarcs)/curcnt, 0
    return zero_cnt(curarcs)/curcnt, zero_cnt(otherarcs)/othcnt


def get_all_arc_avgs(dae_df):
    total = [0, 0]
    cnt = 0
    for index, row in dae_df.iterrows():
        avgs = get_dae_arc_coverage(row)
        if avgs[1]>=0:
            cnt+=1
            total[0]+=avgs[0]
            total[1]+=avgs[1]
    total[0] = float(total[0])/cnt
    total[1] = float(total[1])/cnt
    return total
    
get_all_arc_avgs(dae_prune)
    

[0.5789360736996795, 0.35498937174445727]

In [131]:
tmp = pruned.loc[pruned['dae_pruned'] == '0']
tmp.loc[tmp['halluc']==''].to_csv('no_prune.csv', index=False)

In [97]:
pruned

,doc,full_a,full_b,invalid,short_a,short_b,dae_a,dae_b,precision,recall,f1,oldid,graphind,dae_pruned,halluc,arcs_a,arcs_b
0,"The body of the man in his 70s, was found at t...",A man has been arrested on suspicion of murder...,A man has been arrested on suspicion of murder...,False,home,house,"[0, 0, 1, 1, 1, 1, 1, 1]","[0, 0, 1, 1, 1, 1, 1, 1]",0.995133,0.995132,0.995152,0,0,0,a,{'[CLS] nmod : at [SEP] home [SEP] dead [SEP]'...,{'[CLS] nmod : at [SEP] house [SEP] dead [SEP]...
3,"The body of the man in his 70s, was found at t...",A man has been arrested on suspicion of murder...,A man has been arrested on suspicion of murder...,False,stabbed to death,found dead,"[0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 1, 1, 1, 1]",0.902390,0.931311,0.917078,3,0,a,ab,{'[CLS] nsubjpass [SEP] man [SEP] stabbed [SEP...,{'[CLS] nsubjpass [SEP] man [SEP] found [SEP]'...
4,"The body of the man in his 70s, was found at t...",A man has been arrested on suspicion of murder...,A man has been arrested on suspicion of murder...,False,following the death of a man,after a man was found dead,"[0, 0, 1, 1, 1, 1, 1]","[0, 0, 1, 1, 1, 1, 1, 1]",0.899946,0.900133,0.900441,4,0,0,ab,{'[CLS] nsubjpass [SEP] man [SEP] arrested [SE...,{'[CLS] nsubjpass [SEP] man [SEP] arrested [SE...
5,"The body of the man in his 70s, was found at t...",A man has been arrested on suspicion of murder...,A man has been arrested on suspicion of murder...,False,following the death of a man,after a man was stabbed to death,"[0, 0, 1, 1, 1, 1, 1]","[0, 0, 1, 0, 0, 0, 0, 0]",0.905021,0.888849,0.897318,5,0,b,ab,{'[CLS] nsubjpass [SEP] man [SEP] arrested [SE...,{'[CLS] nsubjpass [SEP] man [SEP] arrested [SE...
6,"The body of the man in his 70s, was found at t...",A man has been arrested on suspicion of murder...,A man has been arrested on suspicion of murder...,False,stabbed,dead,"[0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 1, 1, 1, 1, 1, 1]",0.942055,0.945409,0.943956,6,0,a,ab,{'[CLS] xcomp [SEP] stabbed [SEP] found [SEP]'...,{'[CLS] xcomp [SEP] dead [SEP] found [SEP]': 1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407,Birmingham academies Park View and Nansen Prim...,Two schools at the centre of the socalled Troj...,Two schools at the centre of the socalled Troj...,False,face losing their Ofsted funding.,have been told they face losing their Ofsted f...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.958564,0.836212,0.895914,5457,71,b,ab,{'[CLS] nsubj [SEP] schools [SEP] losing [SEP]...,{'[CLS] nsubjpass [SEP] schools [SEP] told [SE...
5412,Birmingham academies Park View and Nansen Prim...,Two schools at the centre of the socalled Troj...,Two schools at the centre of the socalled Troj...,False,have been told they may,could,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.828208,0.953978,0.889514,5462,71,a,,{'[CLS] nsubjpass [SEP] schools [SEP] told [SE...,{}
5413,Birmingham academies Park View and Nansen Prim...,Two schools at the centre of the socalled Troj...,Two schools at the centre of the socalled Troj...,False,have been told they may,will,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.833829,0.937926,0.884949,5463,71,a,,{'[CLS] nsubjpass [SEP] schools [SEP] told [SE...,{}
5414,Birmingham academies Park View and Nansen Prim...,Two schools at the centre of the socalled Troj...,Two schools at the centre of the socalled Troj...,False,have been told they may,are to,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.844853,0.923623,0.883907,5464,71,a,,{'[CLS] nsubjpass [SEP] schools [SEP] told [SE...,{'[CLS] nsubj [SEP] schools [SEP] are [SEP]': ...
